In [1]:
!pip install opencv-python
!pip install opencv-rolling-ball
!pip install sewar


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from skimage import io, filters, feature, measure, metrics
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import cv2
import os
from cv2_rolling_ball import subtract_background_rolling_ball
from sewar import full_ref
from skimage.metrics import structural_similarity as ssim

#Edge detection
from skimage.filters import roberts, sobel, scharr, prewitt, farid, threshold_multiotsu
from scipy import ndimage as nd
from skimage.filters.rank import entropy
from skimage.morphology import disk
import glob
import shutil
from skimage import io
from IPython.core.debugger import Pdb

In [3]:
def calculate_imageError(img_file, ref_img_file):
    ref_img = cv2.imread(ref_img_file, 1)
    img = cv2.imread(img_file, 1)

    mse_skimg = metrics.mean_squared_error(ref_img, img)
    print("MSE: based on scikit-image = ", mse_skimg)

    psnr_skimg = metrics.peak_signal_noise_ratio(ref_img, img, data_range=None)
    print("PSNR: based on scikit-image = ", psnr_skimg)

    rmse_skimg = metrics.normalized_root_mse(ref_img, img)
    print("RMSE: based on scikit-image = ", rmse_skimg)

    ssim_skimg = ssim(ref_img, img,
                      data_range = img.max() - img.min(), 
                      multichannel = True)
    print("SSIM: based on scikit-image = ", ssim_skimg)

    psnr_img=full_ref.psnr(ref_img, img, MAX=None)

    print("PSNR: peak signal-to-noise ratio = ", psnr_img)

In [4]:
calculate_imageError('D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000001_1.jpg', 'D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000001_0.jpg')

MSE: based on scikit-image =  5502.220717670954
PSNR: based on scikit-image =  10.725423530505376
RMSE: based on scikit-image =  0.31542790461252584
SSIM: based on scikit-image =  0.7452580801477681
PSNR: peak signal-to-noise ratio =  10.725423530505376


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\2444997665.py:14: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,


In [5]:
df_imageErrors = pd.DataFrame(columns=['Methodology', 'Battery_Lid_No', 'Image_File', 'Reference_Image_File', 'MSE','RMSE','PSNR','SSIM'])

In [6]:
def calculateAndSave_imageErrors(img_file, ref_img_file, methodology):
    global df_imageErrors
    
    ref_img = cv2.imread(ref_img_file, 1)
    img = cv2.imread(img_file, 1)

    mse_skimg = metrics.mean_squared_error(ref_img, img)
    rmse_skimg = metrics.normalized_root_mse(ref_img, img)
    psnr_skimg = metrics.peak_signal_noise_ratio(ref_img, img, data_range=None)
    
    ssim_skimg = ssim(ref_img, img,
                      data_range = img.max() - img.min(), 
                      multichannel = True)
    
    img_file_name = os.path.splitext(os.path.basename(img_file))[0]
    ref_img_file_name = os.path.splitext(os.path.basename(ref_img_file))[0]
    battery_lid_no = img_file_name[10:-2]
    battery_lid_no = battery_lid_no.lstrip("0")
        
    df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)

In [7]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Sealing Cycle\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Original Image')
    
df_imageErrors[(df_imageErrors.Methodology == 'Original Image')]

D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000001_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000002_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000004_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000005_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000006_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000007_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000007_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000008_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000008_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000009_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000009_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000010_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000010_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000011_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000011_0.jpg


...

D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000051_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000052_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000053_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000053_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000054_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000054_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000055_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000055_0.jpg
D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000056_1.jpg D:\Volvo\BatteryLid Images Sealing Cycle\BatteryLid000056_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)


,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
0,Original Image,1,BatteryLid000001_1,BatteryLid000001_0,1165.003498,0.174573,17.467531,0.736667
1,Original Image,2,BatteryLid000002_1,BatteryLid000002_0,1342.940702,0.194756,16.850235,0.730658
2,Original Image,3,BatteryLid000003_1,BatteryLid000003_0,1361.914579,0.195656,16.789305,0.729762
3,Original Image,4,BatteryLid000004_1,BatteryLid000004_0,1319.69657,0.190672,16.926063,0.727413
4,Original Image,5,BatteryLid000005_1,BatteryLid000005_0,1265.664015,0.182831,17.107619,0.733919
5,Original Image,6,BatteryLid000006_1,BatteryLid000006_0,1057.311781,0.167743,17.888773,0.749705
6,Original Image,7,BatteryLid000007_1,BatteryLid000007_0,1308.815335,0.187091,16.96202,0.710899
7,Original Image,8,BatteryLid000008_1,BatteryLid000008_0,1373.381234,0.194684,16.752893,0.717333
8,Original Image,9,BatteryLid000009_1,BatteryLid000009_0,1173.905947,0.176178,17.434471,0.746173
9,Original Image,10,BatteryLid000010_1,BatteryLid000010_0,1404.667795,0.202087,16.655067,0.723253


In [8]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Cycle Threshold Mean\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Adaptive Mean Thresholding Without Gabor Filter')
    
df_imageErrors[(df_imageErrors.Methodology == 'Adaptive Mean Thresholding Without Gabor Filter')]

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)


D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000001_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000002_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000004_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000005_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000006_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000007_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000007_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000008_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000008_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000009_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000009_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000010_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000010_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000011_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000011_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

...

D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000045_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000045_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000046_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000046_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000047_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000047_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000048_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000048_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000049_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000049_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000050_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000050_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000051_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000052_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000053_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000053_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000054_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000054_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000055_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000055_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000056_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Mean\BatteryLid000056_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)


,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
56,Adaptive Mean Thresholding Without Gabor Filter,1,BatteryLid000001_1,BatteryLid000001_0,8286.376777,0.396651,8.947157,0.674393
57,Adaptive Mean Thresholding Without Gabor Filter,2,BatteryLid000002_1,BatteryLid000002_0,9796.955992,0.434621,8.219892,0.623179
58,Adaptive Mean Thresholding Without Gabor Filter,3,BatteryLid000003_1,BatteryLid000003_0,9658.991198,0.43448,8.281486,0.648554
59,Adaptive Mean Thresholding Without Gabor Filter,4,BatteryLid000004_1,BatteryLid000004_0,8750.577691,0.406614,8.710436,0.639553
60,Adaptive Mean Thresholding Without Gabor Filter,5,BatteryLid000005_1,BatteryLid000005_0,7589.538761,0.377436,9.32865,0.681849
61,Adaptive Mean Thresholding Without Gabor Filter,6,BatteryLid000006_1,BatteryLid000006_0,10257.384056,0.444802,8.020437,0.599719
62,Adaptive Mean Thresholding Without Gabor Filter,7,BatteryLid000007_1,BatteryLid000007_0,7920.792823,0.387535,9.143117,0.666475
63,Adaptive Mean Thresholding Without Gabor Filter,8,BatteryLid000008_1,BatteryLid000008_0,9207.74763,0.41967,8.48927,0.637097
64,Adaptive Mean Thresholding Without Gabor Filter,9,BatteryLid000009_1,BatteryLid000009_0,7810.406567,0.380615,9.204067,0.66
65,Adaptive Mean Thresholding Without Gabor Filter,10,BatteryLid000010_1,BatteryLid000010_0,10855.574475,0.461886,7.774275,0.587033


In [9]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Adaptive Gaussian Thresholding Without Gabor Filter')
    
df_imageErrors[(df_imageErrors.Methodology == 'Adaptive Gaussian Thresholding Without Gabor Filter')]

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000001_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000002_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000004_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000005_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000006_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000007_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000007_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000008_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000008_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000009_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000009_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000010_1.jpg 

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

...

D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000050_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000050_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000051_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000052_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000053_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000053_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000054_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000054_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000055_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000055_0.jpg
D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000056_1.jpg D:\Volvo\BatteryLid Images Cycle Threshold Gaussian\BatteryLid000056_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
112,Adaptive Gaussian Thresholding Without Gabor F...,1,BatteryLid000001_1,BatteryLid000001_0,5816.671124,0.324801,10.484059,0.723982
113,Adaptive Gaussian Thresholding Without Gabor F...,2,BatteryLid000002_1,BatteryLid000002_0,7071.656229,0.358911,9.635592,0.679651
114,Adaptive Gaussian Thresholding Without Gabor F...,3,BatteryLid000003_1,BatteryLid000003_0,6092.884902,0.332736,10.282574,0.715759
115,Adaptive Gaussian Thresholding Without Gabor F...,4,BatteryLid000004_1,BatteryLid000004_0,6623.158768,0.346681,9.920152,0.686864
116,Adaptive Gaussian Thresholding Without Gabor F...,5,BatteryLid000005_1,BatteryLid000005_0,6080.14912,0.332193,10.291661,0.717973
117,Adaptive Gaussian Thresholding Without Gabor F...,6,BatteryLid000006_1,BatteryLid000006_0,7312.733582,0.36459,9.490006,0.672425
118,Adaptive Gaussian Thresholding Without Gabor F...,7,BatteryLid000007_1,BatteryLid000007_0,6183.452437,0.335079,10.218493,0.706096
119,Adaptive Gaussian Thresholding Without Gabor F...,8,BatteryLid000008_1,BatteryLid000008_0,6985.876269,0.357972,9.688595,0.682626
120,Adaptive Gaussian Thresholding Without Gabor F...,9,BatteryLid000009_1,BatteryLid000009_0,6253.054333,0.336278,10.169882,0.709244
121,Adaptive Gaussian Thresholding Without Gabor F...,10,BatteryLid000010_1,BatteryLid000010_0,8094.416892,0.385964,9.048948,0.647726


In [10]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Cycle Gabor\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Gabor Filter')
    
df_imageErrors[(df_imageErrors.Methodology == 'Gabor Filter')]

D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000001_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000002_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000004_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000005_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000006_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000007_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000007_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000008_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000008_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000009_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000009_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000010_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000010_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000011_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000011_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000012_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000012_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000013_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000013_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000014_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000014_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000015_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000015_0.jpg


...

D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000040_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000040_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000041_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000041_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000042_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000042_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000043_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000043_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000044_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000044_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000045_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000045_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000046_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000046_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000047_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000047_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000048_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000048_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000049_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000049_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000050_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000050_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000051_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor\BatteryLid000052_0.jpg
D:\Volvo

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
168,Gabor Filter,1,BatteryLid000001_1,BatteryLid000001_0,1178.310257,0.161382,17.418207,0.766359
169,Gabor Filter,2,BatteryLid000002_1,BatteryLid000002_0,1448.565166,0.18529,16.521423,0.748163
170,Gabor Filter,3,BatteryLid000003_1,BatteryLid000003_0,1419.161984,0.181823,16.610484,0.753555
171,Gabor Filter,4,BatteryLid000004_1,BatteryLid000004_0,1396.239167,0.181046,16.681205,0.75091
172,Gabor Filter,5,BatteryLid000005_1,BatteryLid000005_0,1293.974441,0.169887,17.011547,0.760453
173,Gabor Filter,6,BatteryLid000006_1,BatteryLid000006_0,1088.211239,0.156864,17.763672,0.766146
174,Gabor Filter,7,BatteryLid000007_1,BatteryLid000007_0,1326.871192,0.171814,16.902516,0.754304
175,Gabor Filter,8,BatteryLid000008_1,BatteryLid000008_0,1469.633548,0.185136,16.458713,0.741807
176,Gabor Filter,9,BatteryLid000009_1,BatteryLid000009_0,1172.49763,0.162883,17.439684,0.770503
177,Gabor Filter,10,BatteryLid000010_1,BatteryLid000010_0,1509.82109,0.191907,16.341549,0.739419


In [11]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Adaptive Mean Thresholding After Gabor Filter')
    
df_imageErrors[(df_imageErrors.Methodology == 'Adaptive Mean Thresholding After Gabor Filter')]

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000001_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000002_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000004_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000005_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000006_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000007_1.jpg

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000008_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000008_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000009_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000009_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000010_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000010_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000011_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000011_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000012_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000012_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000013_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000013_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000014_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000014_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000015_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000015_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000016_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000016_0.jpg


...

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000043_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000043_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000044_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000044_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000045_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000045_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000046_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000046_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000047_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000047_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000048_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000048_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000049_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000049_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000050_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000050_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000051_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000052_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Mean\BatteryLid000053_1.jpg

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
224,Adaptive Mean Thresholding After Gabor Filter,1,BatteryLid000001_1,BatteryLid000001_0,6068.520481,0.333545,10.299975,0.729342
225,Adaptive Mean Thresholding After Gabor Filter,2,BatteryLid000002_1,BatteryLid000002_0,7414.079384,0.368241,9.430231,0.667907
226,Adaptive Mean Thresholding After Gabor Filter,3,BatteryLid000003_1,BatteryLid000003_0,6262.012356,0.339938,10.163664,0.715657
227,Adaptive Mean Thresholding After Gabor Filter,4,BatteryLid000004_1,BatteryLid000004_0,7065.614252,0.359904,9.639304,0.669178
228,Adaptive Mean Thresholding After Gabor Filter,5,BatteryLid000005_1,BatteryLid000005_0,6176.816689,0.336466,10.223156,0.721097
229,Adaptive Mean Thresholding After Gabor Filter,6,BatteryLid000006_1,BatteryLid000006_0,8200.875592,0.387434,8.992201,0.643298
230,Adaptive Mean Thresholding After Gabor Filter,7,BatteryLid000007_1,BatteryLid000007_0,5970.5848,0.331014,10.370635,0.736265
231,Adaptive Mean Thresholding After Gabor Filter,8,BatteryLid000008_1,BatteryLid000008_0,6881.536391,0.355162,9.753949,0.686602
232,Adaptive Mean Thresholding After Gabor Filter,9,BatteryLid000009_1,BatteryLid000009_0,6377.760664,0.341727,10.084121,0.715099
233,Adaptive Mean Thresholding After Gabor Filter,10,BatteryLid000010_1,BatteryLid000010_0,8650.895904,0.39967,8.760193,0.631208


In [12]:
# All unexpanded battery lid sealings
unexp_image_files = glob.glob("D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\*_0.jpg")
unexp_image_files.sort(key=os.path.getmtime)

for unexp_image_file in unexp_image_files:
    exp_image_file = unexp_image_file.replace('_0.jpg', '_1.jpg')
       
    print(exp_image_file, unexp_image_file)
    calculateAndSave_imageErrors(exp_image_file, unexp_image_file, 'Adaptive Gaussian Thresholding After Gabor Filter')
    
df_imageErrors[(df_imageErrors.Methodology == 'Adaptive Gaussian Thresholding After Gabor Filter')]

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_sk

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000001_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000001_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000002_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000002_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000003_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000003_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000004_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000004_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000005_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000005_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000006_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000006_0.jpg


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

...

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000043_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000043_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000044_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000044_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000045_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000045_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000046_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000046_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000047_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000047_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000048_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000048_0.jpg
D:\Volvo\BatteryLid Images C

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000050_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000050_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000051_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000051_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000052_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000052_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000053_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000053_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000054_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000054_0.jpg
D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000055_1.jpg D:\Volvo\BatteryLid Images Cycle Gabor Threshold Gaussian\BatteryLid000055_0.jpg
D:\Volvo\BatteryLid Images C

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.append(pd.DataFrame({'Methodology': methodology, 'Battery_Lid_No': battery_lid_no, 'Image_File': img_file_name, 'Reference_Image_File': ref_img_file_name, 'MSE': mse_skimg, 'RMSE': rmse_skimg, 'PSNR': psnr_skimg, 'SSIM': ssim_skimg}, index=[0]), ignore_index=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:11: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_8868\1142407895.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_imageErrors = df_imageErrors.a

,Methodology,Battery_Lid_No,Image_File,Reference_Image_File,MSE,RMSE,PSNR,SSIM
280,Adaptive Gaussian Thresholding After Gabor Filter,1,BatteryLid000001_1,BatteryLid000001_0,5502.220718,0.315428,10.725424,0.745258
281,Adaptive Gaussian Thresholding After Gabor Filter,2,BatteryLid000002_1,BatteryLid000002_0,6112.049763,0.331436,10.268935,0.725424
282,Adaptive Gaussian Thresholding After Gabor Filter,3,BatteryLid000003_1,BatteryLid000003_0,5158.220041,0.306094,11.005805,0.760744
283,Adaptive Gaussian Thresholding After Gabor Filter,4,BatteryLid000004_1,BatteryLid000004_0,5876.41046,0.325305,10.439682,0.729691
284,Adaptive Gaussian Thresholding After Gabor Filter,5,BatteryLid000005_1,BatteryLid000005_0,5566.764557,0.317722,10.674775,0.74012
285,Adaptive Gaussian Thresholding After Gabor Filter,6,BatteryLid000006_1,BatteryLid000006_0,6738.853927,0.348419,9.844943,0.693826
286,Adaptive Gaussian Thresholding After Gabor Filter,7,BatteryLid000007_1,BatteryLid000007_0,5248.283852,0.308222,10.93063,0.757238
287,Adaptive Gaussian Thresholding After Gabor Filter,8,BatteryLid000008_1,BatteryLid000008_0,5772.883378,0.322931,10.516876,0.73691
288,Adaptive Gaussian Thresholding After Gabor Filter,9,BatteryLid000009_1,BatteryLid000009_0,5467.3348,0.314808,10.753047,0.740194
289,Adaptive Gaussian Thresholding After Gabor Filter,10,BatteryLid000010_1,BatteryLid000010_0,7645.867299,0.371204,9.296536,0.668604


In [13]:
df_imageErrors.to_csv('D:\\Volvo\\ImageErrors.csv')

In [14]:
pd.set_option('expand_frame_repr', False)

df_imageErrors_summary = df_imageErrors.groupby('Methodology').agg({'SSIM': ['mean', 'min', 'max']}) 
  
print("Mean, min, and max values of SSIM group by Methodology") 
print(df_imageErrors_summary)

Mean, min, and max values of SSIM group by Methodology
                                                        SSIM                    
                                                        mean       min       max
Methodology                                                                     
Adaptive Gaussian Thresholding After Gabor Filter   0.718040  0.643746  0.765644
Adaptive Gaussian Thresholding Without Gabor Fi...  0.692787  0.640681  0.757939
Adaptive Mean Thresholding After Gabor Filter       0.688739  0.626135  0.760671
Adaptive Mean Thresholding Without Gabor Filter     0.645456  0.575696  0.746147
Gabor Filter                                        0.752145  0.736330  0.773103
Original Image                                      0.727825  0.706122  0.750634


In [15]:
df_imageErrors_summary.to_csv('D:\\Volvo\\ImageErrorsSummary.csv')